# TOV Sequence Integrator

## Detailed Analysis

This script is used to generate the full Mass-Radius (M-R) sequence for a given Equation of State (EoS). By integrating the Tolman-Oppenheimer-Volkoff (TOV) equations over a wide range of central pressures, the complete stable branch of the neutron star family is constructed.

The script serves as the bridge between the microphysical EoS parameters (inputs) and the macroscopic observables (Mass, Radius, Tidal Deformability). It handles the distinct initialization logic required for both Hadronic models (which have a crust) and Quark models (which have a vacuum pressure and finite surface density).

## Physics and Math

### EoS Initialization
To start the TOV integration, the central energy density $\epsilon_c$ must be determined from the central pressure $P_c$.

**Quark Matter (Generalized CFL):**
The relationship is algebraic. The chemical potential $\mu$ is found by solving the pressure equation:
$$
P = \frac{3}{4\pi^2}\mu^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu^2 - B
$$
Once $\mu$ is found, the energy density is computed:
$$
\epsilon = \frac{3}{4\pi^2}\mu^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu^2 + B
$$
Here, $B$ is the bag constant, $\Delta$ is the gap parameter, and $m_s$ is the strange quark mass.

**Hadronic Matter:**
For hadronic stars, $\epsilon(P)$ is obtained via interpolation of the core model functions (scaled by $\alpha$ and mixed by weight $w$) or from the crust parameterizations if $P_c$ is low.

### Tidal Deformability
The dimensionless tidal deformability $\Lambda$ is calculated from the Love number $k_2$ and the compactness $C$:

$$
C = \frac{GM}{Rc^2}
$$

$$
\Lambda = \frac{2}{3} k_2 C^{-5}
$$

The Love number $k_2$ is derived from the value of the tidal variable $y_{tidal}$ at the stellar surface, utilizing the analytic solution provided by Hinderer et al.

## Code Walkthrough

### 1. Pressure Grid Configuration
The sequence is generated by sweeping through central pressures.
*   **Quark Stars:** A range from $10^0$ to $10^4$ MeV/fm$^3$ is used. Higher pressures are required to overcome the vacuum pressure $B$.
*   **Hadronic Stars:** A wider range ($10^{-3}$ to $10^{3.8}$) is used to capture low-mass stars dominated by the crust.

### 2. Initialization and Sound Speed
Inside the loop, the central boundary conditions are established.
*   **Quark Logic:** The quadratic equation for $\mu^2$ is solved to find $\epsilon_c$. The speed of sound is derived analytically from the derivatives of the pressure equation.
*   **Hadronic Logic:** The appropriate crust or core function is selected based on $P_c$ and the transition pressure $P_{trans}$. Homologous scaling ($\alpha$) and mixing ($w$) are applied here. A "causality clamp" is tracked to ensure $\frac{d\epsilon}{dP} \ge 1$ (i.e., $c_s \le 1$).

### 3. Surface Density Calculation
For Quark stars, the energy density does not vanish at the surface; instead, it drops to a finite value where $P=0$.
*   The equation $P(\mu_{surf}) = 0$ is solved to find $\epsilon_{surf}$.
*   This value is recorded for surface density gap analysis later in the pipeline.

### 4. TOV Integration
The `solve_ivp` function (using the RK45 method) integrates the coupled ODEs provided by `tov_rhs`.
*   **State Vector:** $[Mass, Pressure, y_{tidal}]$.
*   **Termination:** Integration stops when the pressure drops to zero (defined by `surface_event`).

### 5. Post-Processing and Filtering
Once the integration for a specific $P_c$ concludes:
*   **Physicality Checks:** Solutions with extremely small radii or masses are discarded.
*   **Tidal Calculation:** The complex algebraic formula for $k_2$ is evaluated using the surface value of $y_{tidal}$ and compactness $C$.
*   **Stability limit:** The maximum mass ($max\_m$) encountered in the sequence is tracked. If the mass begins to decrease with increasing pressure, the stable branch has ended.

The function returns the accumulated curve data and the maximum stable mass found.